In [ ]:
import requests
import pandas as pd
from config import abuse_API_key

res = requests.get("https://api.abuseipdb.com/api/v2/blacklist", headers={"Key": abuse_API_key})
res.json()

In [ ]:
data = res.json()
ips = []
for i in range (800):
    ips.append(data['data'][i]['ipAddress'])


In [ ]:
detailed = []
i = 0
for address in ips:
    detailed.append(requests.get(f"https://api.abuseipdb.com/api/v2/check?ipAddress={address}",
             headers={"Key": "856a7889e9bc917f2b54aff296d3380cacdea7c240c3a106ae72f0c01d3d1c2d5c1ebdf7cb6e30c2"}).json())



In [ ]:
detailed[0]['data']

In [ ]:
isp_count = {}
for entry in detailed:
    isp_count[entry['data']['domain']] = isp_count.get(entry['data']['domain'], 0) + 1
sorted_isps = sorted(isp_count, reverse=True, key= lambda x: isp_count[x])
counts = {}
for entry in sorted_isps:
    counts[entry] = isp_count[entry]

counts

In [ ]:
isp_count = {}
unique_count = {}
for entry in detailed:
    isp_count[entry['data']['domain']] = isp_count.get(entry['data']['domain'], 0) + 1
    unique_count[entry['data']['domain']] = unique_count.get(entry['data']['domain'], 0) + entry['data']['numDistinctUsers']

ratio = {}
for domain, count in isp_count.items():
    ratio[domain] = count/unique_count[domain]
    
sorted_ratio = dict(sorted(ratio.items(), key= lambda v: v[1], reverse=True))
sorted_ratio

sorted_unique = dict(sorted(unique_count.items(), key= lambda v: v[1]))
# sorted_unique

# users >> abusive IPs -> users are less abusive
# users ~= abusive IPs -> users are more abusive

# abuse rank higher than user rank -> -- (abusive IPs/users >> users)
# abuse rank lower than user rank -> ++ (abusive IPs/users << users)
# equal: insignificant

# ratio_rank = {}
# i = 1
# for e in sorted_ratio:
#     ratio_rank[e] = i
#     i += 1

# users_rank = {}
# i = 1
# for e in sorted_unique:
#     users_rank[e] = i
#     i += 1

# abusive_score = {}
# for domain in ratio_rank:
#     abusive_score[domain] = users_rank[domain] - ratio_rank[domain]
    
# abusive_score = dict(sorted(abusive_score.items(), key= lambda v: v[1], reverse=True))
# abusive_score

del sorted_ratio[None]
for domain in isp_count:
    if isp_count[domain] < 5:
        del sorted_ratio[domain]
df = pd.DataFrame.from_dict(sorted_ratio, orient='index', columns=['Abuse Index'])
df.index.name = "Domain"
df